In [84]:
import torch
import numpy as np
import copy

In [2]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /Users/ldiazbone/.cache/torch/hub/v0.10.0.zip
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/ldiazbone/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:04<00:00, 10.6MB/s]


In [338]:
def permute_weights_layer1(model, perm):
    model_per = copy.deepcopy(model)
    model_per.conv1.weight = torch.nn.Parameter(model.conv1.weight[perm])
    model_per.bn1.weight = torch.nn.Parameter(model.bn1.weight[perm])
    model_per.layer1[0].conv1.weight =  torch.nn.Parameter(model.layer1[0].conv1.weight.transpose(0,1)[perm].transpose(0,1))
    return model_per

In [332]:
perm = torch.tensor(np.random.permutation(np.linspace(0,63,64)),dtype=torch.int)
randimg = torch.rand(1,3,128,128)

In [333]:
model_per = permute_weights_layer1(model, perm)

In [334]:
model_per.conv1(randimg).shape

torch.Size([1, 64, 64, 64])

In [335]:
print(model_per.conv1(randimg)[0][0][0])
print(model.conv1(randimg)[0][perm[0]][0])

tensor([-0.1290,  0.0654,  0.0805, -0.1135, -0.4525, -0.4998, -0.4238, -0.5268,
        -0.4952, -0.4268, -0.3255, -0.2213, -0.2246, -0.3578, -0.3320, -0.4464,
        -0.4389, -0.2673, -0.3609, -0.4273, -0.5187, -0.4803, -0.6745, -0.5098,
        -0.1869, -0.0511, -0.1644, -0.4683, -0.3508, -0.3740, -0.3760, -0.5410,
        -0.3917,  0.0217, -0.0293, -0.2484, -0.4216, -0.0723,  0.0886,  0.1113,
        -0.1648,  0.1411,  0.2137,  0.0754, -0.1194, -0.3785, -0.3782, -0.3739,
        -0.2155, -0.5233, -0.8680, -0.5897, -0.4466,  0.0034, -0.1476, -0.4479,
        -0.4886, -0.2331,  0.2012,  0.0879,  0.1163,  0.1941,  0.0774, -0.0276],
       grad_fn=<SelectBackward0>)
tensor([-0.1290,  0.0654,  0.0805, -0.1135, -0.4525, -0.4998, -0.4238, -0.5268,
        -0.4952, -0.4268, -0.3255, -0.2213, -0.2246, -0.3578, -0.3320, -0.4464,
        -0.4389, -0.2673, -0.3609, -0.4273, -0.5187, -0.4803, -0.6745, -0.5098,
        -0.1869, -0.0511, -0.1644, -0.4683, -0.3508, -0.3740, -0.3760, -0.5410,
     

In [336]:
print(model_per.bn1(model_per.conv1(randimg))[0][0][0])
print(model.bn1(model.conv1(randimg))[0][perm[0]][0])

tensor([0.2000, 0.2453, 0.2488, 0.2036, 0.1247, 0.1137, 0.1314, 0.1074, 0.1147,
        0.1307, 0.1543, 0.1785, 0.1778, 0.1467, 0.1528, 0.1261, 0.1278, 0.1678,
        0.1460, 0.1306, 0.1093, 0.1182, 0.0730, 0.1113, 0.1865, 0.2182, 0.1918,
        0.1210, 0.1484, 0.1430, 0.1425, 0.1041, 0.1388, 0.2351, 0.2232, 0.1722,
        0.1319, 0.2132, 0.2507, 0.2560, 0.1917, 0.2629, 0.2798, 0.2476, 0.2023,
        0.1419, 0.1420, 0.1430, 0.1799, 0.1082, 0.0279, 0.0927, 0.1261, 0.2309,
        0.1957, 0.1258, 0.1163, 0.1758, 0.2769, 0.2505, 0.2572, 0.2753, 0.2481,
        0.2236], grad_fn=<SelectBackward0>)
tensor([ 0.0803,  0.1071,  0.1092,  0.0824,  0.0357,  0.0292,  0.0396,  0.0254,
         0.0298,  0.0392,  0.0532,  0.0676,  0.0671,  0.0487,  0.0523,  0.0365,
         0.0375,  0.0612,  0.0483,  0.0392,  0.0265,  0.0318,  0.0051,  0.0278,
         0.0723,  0.0910,  0.0754,  0.0335,  0.0497,  0.0465,  0.0462,  0.0235,
         0.0441,  0.1011,  0.0940,  0.0638,  0.0399,  0.0881,  0.1103,  0.11

In [337]:
print(model.layer1[0].conv1(model.maxpool(model.bn1(model.conv1(randimg))))[0][0][0])
print(model_per.layer1[0].conv1(model_per.maxpool(model_per.bn1(model_per.conv1(randimg))))[0][0][0])

tensor([-1.0631, -0.7078, -0.7263, -0.6502, -0.7278, -0.5719, -0.5415, -0.7465,
        -0.6088, -0.3878, -0.5736, -0.9092, -0.9408, -0.8433, -0.7619, -0.8790,
        -0.7136, -0.5328, -0.4325, -0.4898, -0.4422, -0.5714, -0.6140, -0.8100,
        -0.6885, -0.5899, -0.4102, -0.5774, -0.8484, -0.7356, -0.7468, -0.8364],
       grad_fn=<SelectBackward0>)
tensor([-0.6986, -0.4793, -0.5471, -0.5145, -0.5563, -0.4077, -0.5144, -0.5974,
        -0.4883, -0.2369, -0.3635, -0.4723, -0.6081, -0.6370, -0.5726, -0.6741,
        -0.5394, -0.3263, -0.2327, -0.2635, -0.2675, -0.3951, -0.4302, -0.4732,
        -0.3264, -0.3224, -0.3004, -0.3838, -0.6457, -0.6200, -0.5591, -0.6542],
       grad_fn=<SelectBackward0>)


In [312]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [303]:
def permute_weights_layer1_test(model, perm):
    model_per = copy.deepcopy(model)
    model_per.conv1.weight = torch.nn.Parameter(model.conv1.weight[perm])
    model_per.bn1.weight = torch.nn.Parameter(model.bn1.weight[perm])
    model_per.conv2.weight =  torch.nn.Parameter(model.conv2.weight.transpose(0,1)[perm].transpose(0,1))
    return model_per

In [350]:
class CN(torch.nn.Module):
    def __init__(self):
        super(CN, self).__init__()
        self.conv1 = torch.nn.Conv2d(3,64,7,2,padding=3, bias=False)
        self.bn1 = torch.nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.maxpool = torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        self.relu = torch.nn.ReLU(inplace=True)
        self.conv2 = torch.nn.Conv2d(64,64,3,1,padding=1, bias=False)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        return x

In [351]:
net = CN()
net.bn1.weight = torch.nn.Parameter(torch.randn(64))
print(net)

CN(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (relu): ReLU(inplace=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
)


In [352]:
net_p = permute_weights_layer1_test(net, perm)

In [353]:
perm

tensor([27, 60, 33, 50, 61, 45,  3, 30, 34, 28, 46, 21,  0, 39, 20, 29, 55, 11,
        23, 40, 63, 10, 36, 19,  9,  6, 41, 59,  2, 12, 25, 53, 42, 22, 49,  4,
        15, 18, 17, 58, 37, 44, 47, 56, 38, 24, 48,  1, 35, 31, 51, 26, 52, 43,
         7, 62, 32, 13, 16,  5, 54, 14, 57,  8], dtype=torch.int32)

In [354]:
randimg2 = torch.rand(1,3,128,128)

In [355]:
torch.isclose(net(randimg2),net_p(randimg2),atol=1e-06)

tensor([[[[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True]],

         [[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True]],

         [[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ...